In [1]:
import cvxpy as cp
import numpy as np
from scipy.linalg import sqrtm
import random as r
import pandas as pd
from qpth.qp import QPFunction
import torch
from sklearn.datasets import make_spd_matrix

In [2]:
global n
n=6       #length of the optimiation variable
global m
m=6       #no.of inequality constraints
global p
p=3       #no.of equality constraints

In [3]:
def find_valid(seeds):
    #find seeds for which the corresponding randomly generated problem yields a solution (from cvxpy)
    seeds_old=seeds.copy()
    seeds_old_old=seeds_old.copy()
    while True:        
        while True: 
            for k,s in enumerate(seeds):
                r.seed(s)
                P=make_spd_matrix(n, random_state=s)
                P = sqrtm(P)
                q=np.array([r.randint(0,9) for i in range(n)]).reshape(n).astype(float)
                G=np.array([r.randint(0,9)*((-1)**r.randint(0,1)) for i in range(m*n)]).reshape(m,n).astype(float)
                h=np.array([0 for i in range(m)]).reshape(m).astype(float)
                A=np.array([r.randint(0,9) for i in range(p*n)]).reshape(p,n).astype(float)
                b=np.array([r.randint(0,9) for i in range(p)]).reshape(p).astype(float)
                try:
                    x,y=qp_cvxpy(n,P,q,G,h,A,b)
                except:
                    #if solving the problem creates error, remove this seed from list
                    seeds.remove(s)
                    continue
                #if the problem is infeasible, y is None; if so remove this seed from list
                if(np.array(y).any()==None):
                    seeds.remove(s)
            if len(seeds_old)==len(seeds):
                break
            else:
                seeds_old=seeds
        if len(seeds_old_old)==len(seeds_old):
            break
        seeds_old_old=seeds_old
#     print("No.of problems generated: ",len(seeds)) 
    return(seeds)

In [4]:
def qp_cvxpy(n,P,q,G,h,A,b):
    x= cp.Variable(n)
    objective=cp.Minimize((1/2)*cp.sum_squares(P*x) + q.T @ x)
    constraints=[G@x <=h,A@x ==b]
    prob=cp.Problem(objective,constraints)
    prob.solve(verbose=False)
    cvxpy_optimum= prob.value
    cvxpy_solution=x.value
    return cvxpy_optimum,cvxpy_solution

In [5]:
#initialize seeds for generating random quadratic problems
seeds=[]
for step in [3,5,7,10,13,17,19,23,29]:
    print(step)
    try_seeds=[i for i in range(10,25000,step)]
    # print(len(try_seeds))
    seeds+=find_valid(try_seeds)
    if(len(set(seeds))>=10000):
        seeds=list(set(seeds))[:10000]
        break
seeds=list(set(seeds))
print("No.of problems generated: ",len(seeds))

3
5
7
10
13
17
19
No.of problems generated:  10000


In [6]:
f=[2,3,4,6]
e=[6,4,5,8,2,1,0]
s= set(f)
s=set(f+e)
list(s)

[0, 1, 2, 3, 4, 5, 6, 8]

In [7]:
columns=["seed"]+[str(i)+"cp" for i in range(n)]
results=pd.DataFrame([],columns=columns)
for k,s in enumerate(seeds):
    r.seed(s)
    P=make_spd_matrix(n, random_state=s)
    q=np.array([r.randint(0,9) for i in range(n)]).reshape(n).astype(float)
    G=np.array([r.randint(0,9)*((-1)**r.randint(0,1)) for i in range(m*n)]).reshape(m,n).astype(float)
    h=np.array([0 for i in range(m)]).reshape(n).astype(float)
    A=np.array([r.randint(0,9) for i in range(p*n)]).reshape(p,n).astype(float)
    b=np.array([r.randint(0,9) for i in range(p)]).reshape(p).astype(float)
    
    P_tch=torch.from_numpy(P)
    q_tch=torch.from_numpy(q)
    G_tch=torch.from_numpy(G)
    h_tch=torch.from_numpy(h)
    A_tch=torch.from_numpy(A)
    b_tch=torch.from_numpy(b)
    
    
    P = sqrtm(P)
    x,y=qp_cvxpy(n,P,q,G,h,A,b)
    re=pd.Series({"seed":s})
    for i in range(n):
        if(y is not None):
            re=re.append(pd.Series({str(i)+"cp":np.round(y[i],6)}))
        else:
            re=re.append(pd.Series({str(i)+"cp":0}))
    
    qpf = QPFunction()
    try:
        solution = qpf(P_tch, q_tch, G_tch, h_tch, A_tch, b_tch)
        qpth_solution=solution.detach().numpy()[0]
    except:
        qpth_solution=[0]*n
    for i in range(n):
        re=re.append(pd.Series({str(i)+"qpt":np.round(qpth_solution[i],6)}))
    results=results.append(re,ignore_index=True )
    


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i

...


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i



--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an 


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an inaccurate and potentially incorrect solution.

Some residual is large.
Your problem may be infeasible or difficult.

You can try using the CVXPY solver to see if your problem is feasible
and you can use the verbose option to check the convergence status of
our solver while increasing the number of iterations.

Advanced users:
You can also try to enable iterative refinement in the solver:
https://github.com/locuslab/qpth/issues/6
--------


--------
qpth warning: Returning an i

In [8]:
error=np.zeros(len(results))
for i in range(n):
    error+=results[str(i)+"cp"]-results[str(i)+"qpt"]
results["cp-qpt-error"]=error    
results["cp-qpt-er_flag"]=error!=0

print("No of solved problems",len(seeds))
print("No of inaccurate results",len(error[error!=0]))

results[results["cp-qpt-er_flag"]]

No of solved problems 10000
No of inaccurate results 441


,seed,0cp,1cp,2cp,3cp,4cp,5cp,0qpt,1qpt,2qpt,3qpt,4qpt,5qpt,cp-qpt-error,cp-qpt-er_flag
4,17.0,-3.547789,-6.946498,9.317247,-10.633284,11.809383,6.198548,-0.005024,-0.859741,2.038402,-2.559575,2.312269,2.031121,3.240155,True
5,19.0,15.902075,-3.555210,-7.195557,-11.752652,4.153690,-6.635221,4.298103,0.976133,-2.610169,-4.747300,1.157036,-3.489779,-4.666899,True
9,28.0,-1.204545,-4.892857,8.237013,12.048701,-4.370130,-20.438312,0.122440,0.414898,0.482550,-0.005486,0.588872,-0.643341,-11.580063,True
20,55.0,-2.181463,9.261561,5.237616,-9.060816,-3.226330,3.304756,0.161856,1.463096,-0.071658,-1.454638,0.118772,0.820593,2.297303,True
53,135.0,3.380764,19.510594,-17.821770,0.172206,-6.386165,10.014125,0.758886,2.954084,0.730931,-0.694544,-1.949890,-0.350863,7.421150,True
64,165.0,95.083761,-27.893529,47.624908,-63.914042,11.313309,-69.655678,9.275558,-0.105376,1.111013,-5.828521,1.819497,-4.134867,-9.578575,True
86,217.0,-3.052560,13.842320,-2.379550,-68.281377,18.695967,55.701405,-1.047920,0.387949,0.573924,-0.903716,-0.016703,1.252142,14.280529,True
99,248.0,12.283770,12.176440,-16.987958,7.002618,-10.360733,-9.309948,1.072039,0.943605,-1.396526,0.073042,0.105792,-0.308810,-5.684953,True
120,297.0,0.165234,-0.786542,10.815701,-22.665421,-2.393271,5.895327,-0.282779,2.384263,-0.971493,1.680650,-0.807869,-0.647630,-10.324114,True
124,307.0,6.817337,-1.753932,0.897864,-4.594644,-2.472074,-3.562477,1.372583,0.102158,0.505125,-0.367717,-0.619595,-0.906169,-4.754311,True


In [9]:
results.to_csv("test_cases.csv")

In [10]:
results.head()

,seed,0cp,1cp,2cp,3cp,4cp,5cp,0qpt,1qpt,2qpt,3qpt,4qpt,5qpt,cp-qpt-error,cp-qpt-er_flag
0,10.0,1.002218,1.064616,0.796417,0.243174,-2.992336,2.006204,1.002218,1.064616,0.796417,0.243174,-2.992336,2.006204,0.000000,False
1,13.0,-1.613671,0.734115,0.851662,0.481962,-0.455964,0.074621,-1.613671,0.734115,0.851662,0.481962,-0.455964,0.074621,0.000000,False
2,15.0,0.334592,-0.228902,-0.245780,0.428199,0.448590,0.404939,0.334592,-0.228902,-0.245780,0.428199,0.448590,0.404939,0.000000,False
3,16.0,1.197429,-0.765534,-3.261708,0.683604,0.119376,1.356290,1.197429,-0.765534,-3.261708,0.683604,0.119376,1.356290,0.000000,False
4,17.0,-3.547789,-6.946498,9.317247,-10.633284,11.809383,6.198548,-0.005024,-0.859741,2.038402,-2.559575,2.312269,2.031121,3.240155,True
